In [55]:
import pandas as pd
from alphabet_detector import AlphabetDetector
import string
import re
import nltk
import numpy as np
import operator
from nltk.stem.snowball import SnowballStemmer

# Data cleaning

In [18]:
path_to_data = '../data/comments1.csv'

In [19]:
df = pd.read_csv(path_to_data)
df = df.dropna()

In [20]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [21]:
def clean(x):
    ad = AlphabetDetector()
    res = x
    for ch in string.punctuation:                                                                                                     
        res = res.replace(ch, ' ')
    res = ''.join([i for i in res if not i.isdigit()])
    res = res.lower()
    res = emoji_pattern.sub(r' ', res)
    res = res.replace('\n', ' ')
    res = res.replace('\t', ' ')
    res = res.replace('\ufeff', ' ')
    res = res.replace('\r\n', '  ')
    res = res.replace('\xa0', ' ')
    res = res.replace('ё', 'е')
    res = re.sub(' +',' ', res)
    if  not ad.only_alphabet_chars(res, "CYRILLIC"): 
        res = ''
    return res

In [22]:
df.text = df.text.apply(clean)
df = df[df.text != '']

In [23]:
#remove comments that were manualy labled
manual = pd.read_csv('../data/manual.csv')
df = df[~df.id.isin(manual.id)]

# Stemming bad words

In [24]:
def stem(word):
    stemmer = SnowballStemmer("russian", ignore_stopwords=True) 
    stemmed_word = stemmer.stem(word)
    if len(stemmed_word) <= 2:
        return word
    return stemmed_word

In [25]:
path_to_bad_words = '../data/bad_words_seed.txt'

In [26]:
bad_words = open(path_to_bad_words).read().split('\n')[:-1]

In [27]:
report_dictionary = [] # array that will contain all bad words
stemmed_dictionary = [] # array that will contain actual stems used for finding bad words

In [30]:
for word in bad_words:
    report_dictionary.append(word)
    stemmed_dictionary.append(stem(word))

In [31]:
stemmed_dictionary = [i for i in stemmed_dictionary if len(i) > 2] 

In [32]:
stemmed_dictionary = list(set(stemmed_dictionary))

In [33]:
stemmed_dictionary.append('хуй') # nltk stemmer can not corectly stem word хуй
stemmed_dictionary.append('хуе')

# Lable coments with bad words

In [34]:
def lable(x):
    global stemmed_dictionary
    tokens = nltk.word_tokenize(x)
    for bad_word in stemmed_dictionary:
        for token in tokens:
            if bad_word in token:
                return True
    return False

In [35]:
df['lable'] = df.text.apply(lable)

# Iterative process of finding new bad words

## Word stemming and counting

In [36]:
all_stemmed = dict()
for sentence in df.text:
    for token in nltk.word_tokenize(sentence):
        word = stem(token)
        if len(word) > 2:
            if word not in all_stemmed:
                all_stemmed[word] = stem(word)

In [37]:
all_counts = dict()
for sentence in df.text:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        word = stem(token)
        if word in all_counts:
            all_counts[word] += 1
        else:
            all_counts[word] = 1

# Likelihood of words being in comments labled as bad or labled as not bad

In [38]:
def likelihood(lable):
    labled_part = dict()
    part = df[df.lable == lable]
    for index, row in part.iterrows():
        sentence = row.text
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            stemed = stem(token)
            if stemed != '':
                if all_counts[stemed] >= 20:
                    if stemed in labled_part:
                        labled_part[stemed] += 1.0/len(part)
                    else:
                        labled_part[stemed] = 1.0/len(part)
    return labled_part

## Criteria functions for decision if word is bad or not bad

In [39]:
def relative_distance(p_good, p_bad):
    difference = dict()
    for key in p_good:
        if key in p_bad:
            difference[key] =(p_bad[key] - p_good[key])/ np.maximum( p_bad[key], p_good[key]) 
    return difference

In [40]:
def logg_odds(p_good, p_bad):
    ratio = dict()
    for key in p_good:
        if key in p_bad:
            odds_good =  p_good[key]/(1 -  p_good[key])
            odds_bad = p_bad[key]/(1- p_bad[key])
            ratio[key] = np.log(odds_bad/odds_good)
    return ratio

## Detecting new bad words

In [41]:
def unstem(stem):
    res = []
    for key in all_stemmed:
        if all_stemmed[key] == stem:
            res.append(key)
    return res    

In [53]:
def sort(x, rev = True):
    return sorted(x.items(), key=operator.itemgetter(1), reverse=rev)

In [42]:
def update_dictionary(word):
    global stemmed_dictionary
    if len(word) <= 2:
        return False
    #for key in stemmed_dictionary:
        #if word in key:
            #return False
    stemmed_dictionary.append(word)
    return True

In [57]:
def iterate(df, report_dictionary, stemmed_dictionary):
    iter = True
    while(iter):
        print('-----------------------------------')
        print('labled as abusive: ', len(df[df.lable == True]))
        #print('List of bad_words')
        #print(report_dictionary)
        p_bad = likelihood(True)
        p_good = likelihood(False)
        prob = relative_distance(p_good, p_bad)
        print(sort(prob)[:10])
        print('New bad words')
        new_words = []
        for key in prob:
            if prob[key] > 0.9:
                if update_dictionary(key):
                    new_words += unstem(key)
                    print( unstem(key))
        report_dictionary += new_words
        if len(new_words) == 0:
            iter = False
            break
        #print('\nNew dictionary')
        #print(report_dictionary)
        df['lable'] =  df.text.apply(relable)
        print('relabled as abusive: ', len(df[df.lable == True]))

In [58]:
iterate(df, report_dictionary, stemmed_dictionary)

-----------------------------------
labled as abusive:  7116
[('колхоз', 0.9532752009245155), ('увелич', 0.9508160009731741), ('финлянд', 0.9409792011678089), ('працюват', 0.9340355777757864), ('корабл', 0.9102883857750697), ('погон', 0.9030893056212173), ('подруг', 0.9003204286389662), ('снижен', 0.8993619712220332), ('гривн', 0.8961671131655898), ('дыряв', 0.8932004592560352)]
New bad words
['погоня', 'погон']
['працюват']
['подруг']
['финлянд']
['увеличен', 'увелич']
['колхоз']
['кораблев', 'корабл']
relabled as abusive:  7209
-----------------------------------
labled as abusive:  7209


KeyboardInterrupt: 

# Evaluation

In [59]:
manual['evaluation'] = manual.text.apply(relable)

In [60]:
tp = 0
tn = 0
fp = 0
fn = 0
for index, row in manual.iterrows():
    if row.label == True and row.evaluation == True:
        tp += 1
    if row.label == False and row.evaluation == False:
        tn += 1
    if row.label == False and row.evaluation == True:
        fp += 1
    if row.label == True and row.evaluation == False:
        #print(row.text)
        fn += 1
accuracy = (tp + fp)/(tp + fp + fn + fp)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1  = 2 * (precision * recall)/(precision + recall)
print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

accuracy:  0.40540540540540543
precision:  0.7277777777777777
recall:  0.3786127167630058
f1:  0.4980988593155893
